In [1]:
import nltk
# Download NLTK data (if not already downloaded)
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
import random
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np

In [12]:
# load dataset for train and test.
df_train = pd.read_csv("/content/drive/MyDrive/REX Technologies/Text Sentiment Analysis/train.csv",
                       encoding="latin-1")
df_test = pd.read_csv("/content/drive/MyDrive/REX Technologies/Text Sentiment Analysis/test.csv",
                      encoding="latin-1")

df_train = df_train[["text", "sentiment"]]
df_test = df_test[["text", "sentiment"]]
df_train = df_train.dropna()
df_test = df_test.dropna()
display(df_train.head())
display(df_test.head())

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [16]:
# Define a function to preprocess the text
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text)
    # Remove stopwords and convert to lowercase
    filtered_words = [word.lower() for word in words if word.isalnum() and word.lower() not in stop_words]
    return " ".join(filtered_words)

# Preprocess the dataset
df_train["text"] = df_train["text"].apply(preprocess_text)
df_test["text"] = df_test["text"].apply(preprocess_text)

X_train = df_train['text'].tolist()
y_train = df_train['sentiment'].tolist()
X_test = df_test['text'].tolist()
y_test = df_test['sentiment'].tolist()

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Transform text data into TF-IDF features
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [17]:
y_test

array([1, 2, 0, ..., 0, 2, 2])

In [18]:
# Train a Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel="linear")
svm_classifier.fit(X_train_tfidf, y_train)

# Predict sentiment labels for the test set
y_pred = svm_classifier.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_rep)

Accuracy: 0.71
Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.64      0.69      1001
           1       0.65      0.77      0.70      1430
           2       0.81      0.71      0.76      1103

    accuracy                           0.71      3534
   macro avg       0.73      0.71      0.71      3534
weighted avg       0.72      0.71      0.71      3534



In [28]:
import time
import sklearn.metrics as sm
from sklearn.utils import all_estimators
import pickle
estimators = all_estimators(type_filter='classifier')
model_name=[]
model_precision=[]
model_accuracy=[]
model_training_time=[]
model_recall = []
model_f1_score = []
model_path = "/content/drive/MyDrive/REX Technologies/Text Sentiment Analysis/Text.model.save.part.2"
for name, get_model in estimators:
    try:
        model = get_model()
        # Train the classifier and measure time
        start_time = time.time()
        model.fit(X_train_tfidf, y_train)
        y_pred = model.predict(X_test_tfidf)
        end_time = time.time()

        # Specify 'macro' average for precision_score
        model_precision.append(sm.precision_score(y_test, y_pred, average='micro'))
        model_accuracy.append(sm.accuracy_score(y_test, y_pred))
        model_f1_score.append(sm.f1_score(y_test, y_pred, average='micro'))
        model_recall.append(sm.recall_score(y_test, y_pred, average='micro'))
        model_name.append(name)
        model_training_time.append(end_time - start_time)

        # Save model with pickle model
        with open(model_path + f"/{name}.pkl", 'wb') as model_file:
            pickle.dump(model, model_file)
    except Exception as e:
        print('Unable to import', name)
        print(e)

Unable to import CategoricalNB
A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.
Unable to import ClassifierChain
_BaseChain.__init__() missing 1 required positional argument: 'base_estimator'
Unable to import GaussianNB
A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.
Unable to import GaussianProcessClassifier
A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.
Unable to import HistGradientBoostingClassifier
A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.
Unable to import LabelPropagation
A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.
Unable to import LabelSpreading
A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.
Unable to import LinearDiscrim

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Unable to import MultiOutputClassifier
MultiOutputClassifier.__init__() missing 1 required positional argument: 'estimator'
Unable to import OneVsOneClassifier
OneVsOneClassifier.__init__() missing 1 required positional argument: 'estimator'
Unable to import OneVsRestClassifier
OneVsRestClassifier.__init__() missing 1 required positional argument: 'estimator'
Unable to import OutputCodeClassifier
OutputCodeClassifier.__init__() missing 1 required positional argument: 'estimator'
Unable to import QuadraticDiscriminantAnalysis
A sparse matrix was passed, but dense data is required. Use X.toarray() to convert to a dense numpy array.
Unable to import RadiusNeighborsClassifier
No neighbors found for test samples array([   7,   13,   26,   29,   37,   64,   79,  123,  166,  167,  172,
        173,  187,  202,  204,  205,  243,  254,  261,  265,  271,  279,
        291,  296,  321,  323,  329,  353,  359,  376,  409,  422,  441,
        448,  466,  473,  494,  495,  497,  503,  508,  510,  51

In [29]:
results=pd.DataFrame({"Model Name":model_name, "Model accuracy":model_accuracy,
                      "Model training time":model_training_time ,"Model precision":model_precision,
                      "Model Recall":model_recall, "Model F1 Score":model_f1_score})
results

,Model Name,Model accuracy,Model training time,Model precision,Model Recall,Model F1 Score
0,AdaBoostClassifier,0.650255,3.302475,0.650255,0.650255,0.650255
1,BaggingClassifier,0.689587,89.797270,0.689587,0.689587,0.689587
2,BernoulliNB,0.677419,0.018501,0.677419,0.677419,0.677419
3,CalibratedClassifierCV,0.689304,2.583194,0.689304,0.689304,0.689304
4,ComplementNB,0.653367,0.013962,0.653367,0.653367,0.653367
5,DecisionTreeClassifier,0.647425,10.918006,0.647425,0.647425,0.647425
6,DummyClassifier,0.404641,0.002544,0.404641,0.404641,0.404641
7,ExtraTreeClassifier,0.616016,0.966415,0.616016,0.616016,0.616016
8,ExtraTreesClassifier,0.712507,84.389503,0.712507,0.712507,0.712507
9,GradientBoostingClassifier,0.641483,25.457916,0.641483,0.641483,0.641483


[]